In [147]:
import pandas as pd

df = pd.read_csv('confectionery.csv')
df_count = pd.DataFrame(df.groupby('Item')['Item'].count()) 
df_new = pd.DataFrame(df_count['Item'].sort_values(ascending = False).head())
df_new = df_new.append(pd.DataFrame(df_count['Item'].sort_values().head()))
df_new.drop(df_new.columns[[0]], axis = 1, inplace = True)
df_new.to_csv("output1.csv", header = False)

df['TimeSlot'] = pd.to_datetime(df['Datetime']).dt.hour
df['TimeSlot'] = df['TimeSlot'].astype(str) + "-" + df['TimeSlot'].add(1).astype(str)
df_hrs = pd.DataFrame(df.groupby('TimeSlot')['Item'].count().sort_values(ascending = False))
time = list(df_hrs.index.values)


dayOfWeek = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['Weekday'] = pd.to_datetime(df['Datetime']).dt.dayofweek.map(dayOfWeek)
df_weekday= pd.DataFrame(df.groupby('Weekday')['Item'].count().sort_values(ascending = False))
day = list(df_weekday.index.values)

with open("output2.csv", mode = "w") as f:
    f.write(str(time[0]))
    f.write('\n')
    f.write(str(day[0]))

In [31]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv('german_data.csv')
df['BillAmount'] = df['UnitPrice'].multiply(df['Quantity']).astype(float)
df_cst = pd.DataFrame(df.groupby('CustomerID')['BillAmount'].sum().sort_values(ascending = False))
CstID = list(df_cst.index.values)

# Removing spaces from the description
df['Description'] = df['Description'].str.strip()
#Dropping rows without invoice number
df.dropna(axis = 0, subset = ['InvoiceNo'], inplace = True)
# Removing the credit transaction with invoice numbers containing C
df = df[~df['InvoiceNo'].str.contains('C')]

basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
def encode_units(x):
    if x <= 0:
        return 0    
    if x >= 1:
        return 1
basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace = True, axis = 1)

#Generating frequent item sets that have a support of at least 5%
frequent_itemsets = apriori(basket_sets, min_support = 0.05, use_colnames = True)

#Generating the rules with their corresponding support, confidence and lift
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1).sort_values(by = 'lift', ascending = False)
result = rules["antecedents"]
res = str(result[13]).replace('frozenset', '').replace('})', '').replace('({', '').replace("'", "")

with open("output.csv", mode = "w") as f:
    f.write(str(CstID[0]))
    f.write('\n')
    f.write(str(res))

In [6]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

df = pd.read_csv('chemical.csv')
df['Timestamp']=pd.to_datetime(df['Timestamp'])

#Perform Dickey-Fuller test:
dftest = adfuller(df.Chemicalconcentration, autolag = 'AIC')
p_value = round(dftest[1], 2)
if (p_value < 0.05):
    var = 'stationary'
else:
    var = 'non-stationary'

model = ARIMA(df.Chemicalconcentration, order = (1, 0, 1)) # order = (p,d,q)
results_ARIMA = model.fit()
rmse = round(np.sqrt(mean_squared_error(df.Chemicalconcentration , results_ARIMA.fittedvalues)), 5)

with open("output.csv", mode = "w") as f:
    f.write(str(p_value))
    f.write('\n')
    f.write(str(var))
    f.write('\n')
    f.write(str(rmse))